In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [4]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

In [6]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(3,3)
    def forward(self,x):
        return self.linear(x)
    
model=SoftmaxClassifierModel()

optimizer=optim.SGD(model.parameters(),lr=0.1)

def train(model,x_train,y_train,optimizer):
    
    nb_epochs=1000
    
    for epoch in range(nb_epochs+1):
        
        prediction=model(x_train)
        cost=F.cross_entropy(prediction,y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        if epoch%100==0:
            print("Epochs:{:4d}/{}Cost:{:.6f}".format(
            epoch,nb_epochs,cost.item()
            ))
            
def test(model, optimizer, x_test, y_test):
    
    prediction = model(x_test)
    
    predicted_classes = prediction.max(1)[1]
    
    correct_count = (predicted_classes == y_test).sum().item()
    
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

Zero-centerd&Normalization

In [7]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [10]:
#对每一列（每个特征）进行标准化
mu=x_train.mean(dim=0)
sigma=x_train.std(dim=0)
#sigma is the standard deviation and mu is the mean.
norm_x_train=(x_train-mu)/sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


Overfitting

It may overfit the training data and perform poorly on test data.

There are three main ways to prevent this:

More training data

Fewer features

Regularization:not have too big numbers in the weights

In [9]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # Calculate H(x)
        prediction = model(x_train)

        # Calculate cost
        cost = F.mse_loss(prediction, y_train)
        
        #（L2 正则化）
        # Calculate L2 norm
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
        #其中 λ（正则化系数）控制 L2 正则化的强度（此处为1）
        cost += l2_reg

        # Improve H(x) based on cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))
